<h1 style='font-size:40px'> Ensemble Learning and Random Forests</h1>
<div>
    <ul style='font-size:20px'> 
        <li> 
            O aprendizado Ensemble consiste em fazer previsões embasadas em um conjunto de modelos, ao invés de um único. Essa técnica é inspirada no conceito de <em> sabedoria da multidão</em>.
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Voting Classifiers</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Um classificador por voto é um conjunto de classificadores diferentes que operam como um só. A previsão final é a mais recorrente entre cada um dos algoritmos individuais.
        </li>
    </ul>
</div>

In [15]:
from warnings import filterwarnings
filterwarnings('ignore')

In [24]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import recall_score
from sklearn.ensemble import VotingClassifier

X, y = load_breast_cancer(return_X_y=True)

log_reg = LogisticRegression()
knn = KNeighborsClassifier()
naive_bayes = GaussianNB()

voting_clf = VotingClassifier([
    ('log_reg', log_reg),
    ('knn', knn),
    ('naive_bayes', naive_bayes)
], voting='hard')

for clf in (log_reg, knn, naive_bayes, voting_clf):
    y_pred = clf.fit(X,y)
    y_pred = clf.predict(X)
    print(clf.__class__.__name__, recall_score(y, y_pred))

LogisticRegression 0.969187675070028
KNeighborsClassifier 0.9747899159663865
GaussianNB 0.9719887955182073
VotingClassifier 0.9859943977591037


<div>
    <ul style='font-size:20px'> 
        <li> 
            Podemos definir o valor do argumento "voting" como soft, caso todos os algoritmos usados possam retornar as probabilidades de classe para cada instância. Isso faz com que as probabilidades de cada classe entre os algoritmos tenham as suas médias calculadas. Ao final, a categoria com a maior probabilidade média será aquela prevista.
        </li>
        <li> 
            Alguns classificadores, como o SVC, retornam as probabilidades apenas se o argumento "probability" estiver como True. 
        </li>
    </ul>
</div>

<h2 style='font-size:30px'> Bagging and Pasting</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Bagging e Pasting representam dois tipos de aprendizado em conjunto. Nos seus casos, várias instâncias de um mesmo algoritmo são treinadas em partições aleatórias do dataset.
        </li>
        <li> 
            No Bagging, a repetição de uma dada instância do conjunto de treino é substituída por outra. Em pasting, duplicatas são permitidas.
        </li>
        <li> 
            Em classificação, os ensembles elegem a classe mais prevista entre os modelos individuais. Em regressão, a média das previsões é computada.
        </li>
    </ul>
</div>

<h3 style='font-size:30px;font-style:italic'> Out-of-Bag Evaluation</h3>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Cada previsor que compõe um ensemble é treinado em uma porção restrita do dataset de treino. As instâncias que não o alimentam são denominadas de instâncias out-of-bag (oob). 
        </li>
        <li> 
            O objeto BaggingClassifier nos permite que cada previsor seja avaliado entre suas oob (oob_score=True), nos fornecendo assim uma validação antecipada do ensemble. Ao final, poderemos extrair a acurácia média obtida.
        </li>
    </ul>
</div>

In [28]:
# O oob Evaluation é apenas possível em classificações 'bagging'. Por isso, sette o argumento 'bootstrap' como True.
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
#n_jobs informa ao Python o número de cores do CPU para serem usados no treinamento e previsões.
bag_clf = BaggingClassifier(tree_clf, n_estimators=500, bootstrap=True, oob_score=True, n_jobs=-1)
bag_clf.fit(X,y)

# A acurácia média do Bagging Classifier foi de 96.3%
bag_clf.oob_score_

0.9630931458699473

In [32]:
# 'oob_decision_function' neste caso retorna as probabilidades de classe para cada instância.
bag_clf.oob_decision_function_

array([[0.84656085, 0.15343915],
       [0.98305085, 0.01694915],
       [1.        , 0.        ],
       ...,
       [0.97159091, 0.02840909],
       [1.        , 0.        ],
       [0.0060241 , 0.9939759 ]])

<h3 style='font-size:30px;font-style:italic'>Random Patches and Random Subspaces </h3>
<div>
    <ul style='font-size:20px'> 
        <li> 
            Assim como em Random Forest, podemos definir uma quantidade máxima de features que cada estimador poderá analisar. Dessa maneira, obteremos uma diversidade ainda maior em nosso ensemble. 
        </li>
    </ul>
</div>

In [34]:
# Para fazer isso, sette 'max_features' como um float ou int e 'bootstrap_features' como True.
bag_clf_features = BaggingClassifier(tree_clf, n_estimators=250, max_features=2, bootstrap_features=True, oob_score=True)
bag_clf_features.fit(X,y)

# Infelizmente, essa estratégia foi um pouco pior do que a anterior. Mas vale a pena a considerarmos em nossos projetos!
bag_clf_features.oob_score_

0.945518453427065

<h2 style='font-size:30px'> Random Forests</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
            O Random Forests é um algoritmo de Bagging voltado às Árvores de Decisão. A necessidade de se ter um objeto próprio surgiu da aleatoriedade de formação das árvores e de sua tendência a se viciarem ao dataset de treino.
        </li>
    </ul>
</div>

In [39]:
from sklearn.ensemble import RandomForestClassifier

# O objeto RandomForestClassfier possui tanto argumentos do DecisionTreeClassifier, quanto do BaggingClassifier.
rnd_clf = RandomForestClassifier(n_estimators = 1000, min_impurity_decrease=0.05, max_features=3,n_jobs=-1)
rnd_clf.fit(X,y)
rnd_clf.score(X,y)

0.9420035149384886

<h2 style='font-size:30px'> Extra-Trees</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
           O algoritmo de Extra-Trees confere ainda mais aleatoriedade ao ensemble. Dessa vez, os thresholds utilizados na criação dos nós também são escolhidos de maneira aleatória. Isso torna o modelo mais rápido do que o Random Forests, já que ele não precisa perder tempo computando o melhor threshold para os splits.
        </li>
        <li> 
            Vale lembrar que os Extra-Trees funcionam tanto para classificação, quanto para regressão.
        </li>
    </ul>
</div>

In [42]:
from sklearn.ensemble import ExtraTreesClassifier
extra_clf = ExtraTreesClassifier(n_estimators=1000,max_features=3, max_leaf_nodes=7)
extra_clf.fit(X,y)

# E veja! Obtivemos um score ainda melhor do que o último Random Forest criado.
extra_clf.score(X,y)

0.9507908611599297

<h2 style='font-size:30px'> Feature Importance</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
           Outra enorme qualidade dos objetos de ensemble com Árvores de Decisão é a existência do atributo "feature_importances_". Ele apresenta o grau de relevância de cada feature do dataset para o algoritmo. Isso, por sua vez, é calculado com base na redução do grau de impureza que o uso de tal feature acarreta.
        </li>
        <li> 
            O uso desse atributo pode ser útil em tarefas de limpeza dos DataFrames.
        </li>
    </ul>
</div>

In [43]:
# Observando o 'feature_importances_' do Extra-Trees feito.
extra_clf.feature_importances_

array([0.06026044, 0.01538051, 0.07240843, 0.05797074, 0.00934074,
       0.03516164, 0.05926578, 0.08247204, 0.00772602, 0.00257745,
       0.03292132, 0.00070543, 0.02685925, 0.03482733, 0.00101769,
       0.00621144, 0.00841838, 0.0072371 , 0.00142528, 0.00158883,
       0.07618602, 0.02329403, 0.07440795, 0.07211935, 0.02041379,
       0.03340561, 0.06021188, 0.09254465, 0.01607732, 0.00756358])

<h2 style='font-size:30px'> Boosting</h2>
<div>
    <ul style='font-size:20px'> 
        <li> 
           Os algoritmos de Boosting representam uma outra natureza de ensemble. Nela, os previsores são treinados em sequência, com um tentando corrigir os defeitos de seu antecessor.                                                                              
        </li>
    </ul>
</div>

<p style='color:red'> AdaBoost